# Manager -> **Query pre process**

In [2]:
# Iniciando componentes

import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")
from src.components.memory.memory import Memory
from src.components.collector.collector import AppDataCollector, LLMResponseCollector
from src.components.models.llms.llms import Langchain_OpenAI_LLM, HF_Llama38b_LLM
from src.components.models.embeddings.embeddings import HF_MultilingualE5_Embeddings, Openai_Embeddings
from src.components.memory.memory import Memory
from src.components.memory.memory_interfaces import AIMessage, HumanMessage
from typing import List, Union

# Iniciando modelos LLM
langchain_llm = Langchain_OpenAI_LLM()
langchain_llm.init_model()
hf_llm = HF_Llama38b_LLM()
hf_llm.init_model()

# Iniciando modelos Embeddings
mle5_embeddings = HF_MultilingualE5_Embeddings()
mle5_embeddings.init_model()

openai_embeddings = Openai_Embeddings()
openai_embeddings.init_model()

# Iniciando memoria
messages: List[Union[AIMessage, HumanMessage]]  = []
memory = Memory(messages)


In [3]:
collector = AppDataCollector()
llm_collector = LLMResponseCollector()

In [4]:
last_user_message = memory.add_user_message("I want information from firmware EMED-2012-LR-P12")
collector.current_conversation_data.last_user_message = last_user_message

In [ ]:
# Identificando el user intent y clasificando como simple o complex

from src.app.pipeline_processes.query_pre_process.manager import query_pre_process
import traceback

try:
    r = query_pre_process(hf_llm, memory, collector, llm_collector)
except Exception as e:
    print(e)
    traceback.print_exc()
    

In [ ]:
# Manejando un request simple

from src.app.pipeline_processes.query_pre_process.manager import simple_request_process
import traceback

collector.user_request = "The user is introducing themselves."
collector.request_type = "simple"

try:
    r = simple_request_process(
        llm=hf_llm,
        collector=collector,
        llm_collector=llm_collector,
    )
except Exception as e:
    print(e)
    traceback.print_exc()

In [ ]:
# Manejando un request complejo. Parte 1:

from src.app.pipeline_processes.sql_pre_process.manager import (
    complex_request_process_modification,
)
import traceback

collector.current_conversation_data.current_conversation_summary = "The user wants information from firmware EMED-2012-LR-P12."
collector.current_conversation_data.current_slots = "firmware=EMED-2012-LR-P12"
collector.user_request="The user wants to get information about the firmware EMED-2012-LR-P12."

try:
    r = complex_request_process_modification(
        llm=hf_llm,
        embeddings=mle5_embeddings,
        memory=memory,
        collector=collector,
        llm_collector=llm_collector,
    )
except Exception as e:
    print(e)
    traceback.print_exc()

In [ ]:
# Manejando un request complejo. Parte 2:

from src.app.pipeline_processes.sql_pre_process.manager import (
    complex_request_process_semantics
)
import traceback

collector.current_conversation_data.current_conversation_summary = (
    "The user wants information from firmware EMED-2012-LR-P12."
)
collector.current_conversation_data.current_slots = "firmware=EMED-2012-LR-P12"
collector.user_request = (
    "The user wants to get information about the firmware EMED-2012-LR-P12."
)

try:
    r = complex_request_process_semantics(
        llm=hf_llm,
        embeddings=openai_embeddings,
        collector=collector,
        llm_collector=llm_collector,
    )
except Exception as e:
    print(e)
    traceback.print_exc()

In [10]:
# Manejando un request complejo. Parte 3:

from src.app.pipeline_processes.sql_generation_process.manager import (
    complex_request_sql_generation,
)
import traceback

collector.current_conversation_data.current_conversation_summary = (
    "The user wants information from firmware EMED-2012-LR-P12."
)
collector.current_conversation_data.current_slots = "firmware=EMED-2012-LR-P12"
collector.user_request = (
    "The user wants to get information about the firmware EMED-2012-LR-P12."
)
collector.modified_user_request = (
    "The user wants to get information about the firmware EMED-2012-LR-P12."
)

collector.semantic_info = {
    "fcs_firmware": [
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer firmware",
            1,
            ("fcs_computadores", "fcs_firmware"),
        ),
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer firmware",
            1,
            "fcs_firmware",
        ),
        (
            "Firmware",
            "VARCHAR(100)",
            "Name or version of the firmware",
            2,
            "fcs_firmware",
        ),
        (
            "Estado",
            "VARCHAR(10)",
            "Status of the firmware, active ('ACT') or inactive ('INA')",
            3,
            "fcs_firmware",
        ),
        (
            "IdTipo_Computador",
            "INT",
            "Foreign key linking firmwares with flow computer types",
            999,
            ("fcs_firmware", "fcs_tipo_computador"),
        ),
    ],
    "fcs_tipo_computador": [
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer type",
            1,
            ("fcs_firmware", "fcs_tipo_computador"),
        ),
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer type",
            1,
            "fcs_tipo_computador",
        ),
        (
            "Nombre",
            "VARCHAR(50)",
            "Name or designation of the flow computer type (e.g., OMNI, KHRONE, S600, FC302, FLOWBOSS, F407, F107, Thermofischer)",
            2,
            "fcs_tipo_computador",
        ),
        (
            "Estado",
            "VARCHAR(10)",
            "Status of the computer flow type, active ('ACT') or inactive ('INA')",
            3,
            "fcs_tipo_computador",
        ),
    ],
}

collector.terms_dictionary = [
    {
        "original_term": "firmware",
        "cleaned_term": "firmware",
        "definitions": [
            {
                "sql_instructions": "",
                "table_name": "fcs_firmware",
                "standard_term": "firmware",
                "definition": "Firmware is known as the version of the flow computer.",
                "replace_instruction": "Use 'firmware' instead of version.",
            }
        ],
    },
    {
        "original_term": "EMED-2012-LR-P12",
        "cleaned_term": "emed lrp",
        "definitions": [
            {
                "sql_instructions": "",
                "table_name": "med_sistema_medicion",
                "standard_term": "measurement system",
                "definition": "When talking about the term 'EMED' or similar, it refers to the name/tag of the measurement system.",
                "replace_instruction": "When talking about 'EMED' as a measurement system replace with 'measurement system' instead.",
            },
            {
                "sql_instructions": "When context is related to flow computer firmware, 'EMED-010' could be the name of the flow computer firmware.",
                "table_name": "fcs_firmware",
                "standard_term": "firmware",
                "definition": "Terms related to 'EMED-010' sometimes are names of flow computers firmwares/type",
                "replace_instruction": "When talking about 'EMED-010' as a flow computer firmware, replace with 'firmware' instead of EMED-010.",
            },
            {
                "sql_instructions": "",
                "table_name": "fcs_computadores",
                "standard_term": "flow computer",
                "definition": "When talking about the term 'EMED' or similar, it refers to the name of flow computer.",
                "replace_instruction": "When talking about 'EMED' as a flow computer replace with 'flow computer' instead.",
            },
        ],
    },
]

try:
    r = complex_request_sql_generation(
        llm=hf_llm, collector=collector, llm_collector=llm_collector
    )
except Exception as e:
    print(e)
    traceback.print_exc()

generate-sql
{'text': "\nsql_query:  SELECT fcs_firmware.Id, fcs_firmware.Firmware, fcs_tipo_computador.Nombre FROM dbo_v2.fcs_firmware INNER JOIN dbo_v2.fcs_tipo_computador ON fcs_firmware.IdTipo_Computador = fcs_tipo_computador.Id WHERE fcs_firmware.Firmware = 'EMED-2012-LR-P12';\nsuggestion: The user could improve the natural language question by specifying the desired columns to be retrieved, to avoid unnecessary data retrieval.\nused_tables: fcs_firmware, fcs_tipo_computador"} 




In [ ]:
print(len(llm_collector.llm_responses))
print(llm_collector.llm_responses[6].prompt)

# print(collector.conversation_summary)

In [2]:
collector.user_request = (
    "The human is requesting information from the FQI-EMED-3135.01-067 flow computer."
)
collector.modified_user_request = (
    "The human is requesting information from the FQI-EMED-3135.01-067 flow computer."
)
collector.flavored_request_for_semantic_search = (
    '"The human is requesting information from the firmware."'
)
collector.semantic_info = {
    "fcs_firmware": [
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer firmware",
            1,
            "fcs_firmware",
        ),
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer firmware",
            1,
            ("fcs_computadores", "fcs_firmware"),
        ),
        (
            "Firmware",
            "VARCHAR(100)",
            "Name or version of the firmware",
            2,
            "fcs_firmware",
        ),
        (
            "Estado",
            "VARCHAR(10)",
            "Status of the firmware, active ('ACT') or inactive ('INA')",
            3,
            "fcs_firmware",
        ),
        (
            "IdTipo_Computador",
            "INT",
            "Foreign key linking firmwares with flow computer types",
            999,
            ("fcs_firmware", "fcs_tipo_computador"),
        ),
    ],
    "fcs_computadores": [
        (
            "Id",
            "INT",
            "Unique identifier for each flow computer",
            1,
            "fcs_computadores",
        ),
        (
            "IP",
            "VARCHAR(100)",
            "Flow computer IP address of the flow computer",
            2,
            "fcs_computadores",
        ),
        (
            "Puerto",
            "INT",
            "Flow computer primary port used for communications",
            3,
            "fcs_computadores",
        ),
        ("Tag", "VARCHAR(100)", "Flow computer name", 4, "fcs_computadores"),
        (
            "Estado",
            "VARCHAR(100)",
            "Current status of the flow computer, active ('1') or inactive ('2')",
            5,
            "fcs_computadores",
        ),
        (
            "IdFirmware_fk",
            "INT",
            "Foreign key linking flow computers with respective firmware",
            999,
            ("fcs_computadores", "fcs_firmware"),
        ),
    ],
}

collector.sql_code=("""

SELECT c.Tag, f.Firmware AS 'Revision/Version', tc.Nombre AS Marca FROM dbo_v2.fcs_computadores c JOIN dbo_v2.fcs_firmware f ON f.Id = c.IdFirmware_fk JOIN dbo_v2.fcs_tipo_computador tc ON tc.Id = f.IdTipo_Computador_fk WHERE c.Tag = 'FQI-EMED-3135.01-067'

""").strip()

collector.assistant_sql_code_suggestion = 'Please replace the placeholder "<Flow computer Tag>" with the actual tag of the flow computer you are interested in.'

collector.assistant_sql_code_analysis='The query is almost complete and ready for execution, but it needs one more piece of information, "Flow computer Tag", to be specified.'

collector.ai_pre_response = "The FQI-EMED-3135.01-067 flow computer is a FB107 revision/version manufactured by FLOWBOSS."

collector.ai_post_response = "El computador de flujo FQI-EMED-3135.01-067 es una revisión/versión FB107 fabricada por FLOWBOSS."



In [4]:
#  """Para pre procesar, obtener el request y clasificacion del request se usa: """




"""Si el request es simple, se usa: """
# simple_request_process(langchain_llm, memory, collector)



"""Si el request es complejo se usa: """
# ? Para modificar el request:
# complex_request_process_modification(hf_llm, mle5_embeddings, collector)


# ? Para buscar tablas e informacion semantica de la base sql en la base vectorial
# complex_request_process_semantics(langchain_llm, openai_embeddings, collector)


# ? Para generar el sql en primera instancia
# complex_request_sql_generation(langchain_llm, collector)

# ? Para verificar la clasificacion del sql completo o incompleto
# complex_request_sql_verification(hf_llm, collector)

# ? Para generando un pre query si el sql es incompleto
# complex_request_pre_query_generation(langchain_llm, collector)


# ? Para generar la respuesta de resumen al dtaframe del sql
# primero se debe ejecutar el sql para obtener el dataframe df
# df = run_sql(collector.sql_code) o run_sql(collector.sql_pre_query)
# complex_request_sql_summary_response(langchain_llm, collector, df)

"""Post procesando la respuesta: """
# query_post_process(langchain_llm, collector, "Quiero saber informacion de FQI-EMED-3135.01-067")


'Post procesando la respuesta: '